In [2]:
import pandas as pd
import numpy as np
%load_ext rpy2.ipython

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%%R
data = read.csv("/content/drive/MyDrive/BUDT758B/Data/cleaned_datafinal.csv", encoding = 'utf-8')


In [20]:
%%R
require("tidyverse")
require("xgboost")

R[write to console]: Loading required package: xgboost



In [12]:
%%R
data <- data %>% 
  select(-c(id_artists, name, release_date, year, popularity, is_popular, artist1, artist2, artist3, artist4, X)) %>% 
  filter(!is.na(data$decade))

R[write to console]: Error: Can't subset columns that don't exist.
✖ Column `id_artists` doesn't exist.
Run `rlang::last_error()` to see where the error occurred.

R[write to console]: In addition: 
R[write to console]: Warning message:

R[write to console]: In library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE,  :
R[write to console]: 
 
R[write to console]:  there is no package called ‘xgboost’




Error: Can't subset columns that don't exist.
✖ Column `id_artists` doesn't exist.
Run `rlang::last_error()` to see where the error occurred.


In [10]:
%%R
colnames(data)

 [1] "duration_ms"      "explicit"         "danceability"     "energy"          
 [5] "key"              "loudness"         "mode"             "speechiness"     
 [9] "acousticness"     "instrumentalness" "liveness"         "valence"         
[13] "tempo"            "time_signature"   "n_artists"        "decade"          


In [13]:
%%R
data$decade <- as.numeric(data$decade)

In [14]:
%%R 
data<- data %>%
       mutate(decade = case_when(decade == 1920 ~1,
                        decade == 1930 ~2,
                        decade == 1940 ~3,
                        decade == 1950 ~4,
                        decade == 1960 ~5,
                        decade == 1970 ~6,
                        decade == 1980 ~7,
                        decade == 1990 ~8,
                        decade == 2000 ~9,
                        decade == 2010 ~10,
                        decade == 2020 ~11))

In [16]:
%%R
train_split = sample(nrow(data), .7*nrow(data))
train = data[train_split,]
test = data[-train_split,]

train.X <- train %>%
            select(-decade)
train.y <- as.data.frame(train$decade)

test.X <- test %>%
  select(-decade)
test.y <- as.data.frame(test$decade)





table(data$decade)


     1      2      3      4      5      6      7      8      9     10     11 
  7610  13037  18042  35370  47270  61841  82322 108875  86841 105245  20218 


In [17]:
%%R 
train_spot_matrix <-  data.matrix(train.X)
train_y_matrix <- as.matrix(train.y)
train_y_matrix <- train_y_matrix - 1
train_y_matrix <- as.numeric(train_y_matrix)



unique(train.y$`train$decade`)

 [1]  5  8  7  9 10  3 11  4  2  6  1


In [18]:
%%R
train.y$class_numeric<-as.numeric(train.y$`train$decade`)
train.y<-train.y%>%mutate(class_numeric=class_numeric-1)
table(train.y)


length()
as.numeric(train.y$`train$decade`)

R[write to console]: Error in length() : 0 arguments passed to 'length' which requires 1




Error in length() : 0 arguments passed to 'length' which requires 1


In [19]:
%%R
xg_model<- xgboost(data = train_spot_matrix, label = as.numeric(train.y$class_numeric), max.depth = 6, eta = 1 , nthread = 4,  lambda = 1, 
                   nrounds = 1000, num_class = 11,
                   objective = "multi:softmax", eval_metric="mlogloss", prediction = TRUE )






R[write to console]: Error in xgboost(data = train_spot_matrix, label = as.numeric(train.y$class_numeric),  : 
  could not find function "xgboost"




Error in xgboost(data = train_spot_matrix, label = as.numeric(train.y$class_numeric),  : 
  could not find function "xgboost"


In [ ]:
%%R
preds <- predict(xg_model, newdata = data.matrix(test.X))

preds <- as.data.frame(preds)

classification <- ifelse(preds$preds == test.y$`test$decade`,1,0)
mean(classification)
#Log Loss is the most important classification metric based on probabilities. ... For any given problem, a lower log-loss value 
#means better predictions. 
#Log Loss is a slight twist on something called the Likelihood Function.
